In [18]:
import json
from pathlib import Path 
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import hashlib

In [4]:
es_client = Elasticsearch(['http://localhost:9200'])
es_client.info()

ObjectApiResponse({'name': '5600fc87b7bb', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Jv1hLysGRsKs8pNv9bNkfg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/taras/.cache/pypoetry/virtualenvs/src-SOtueW5z-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
with open(Path().cwd().parents[0]/'data'/'initial_data.json', 'r') as f:
    data = json.load(f)

In [19]:
data

[{'vague': 'Can we look into the tech stuff that isn’t firing right?',
  'actual': 'We need to investigate the issues causing the deployment failures.'},
 {'vague': "Something's off with the software magic we deploy every now and then.",
  'actual': 'We need to address the stability issues in our deployment environments.'},
 {'vague': 'I feel like the codes are having a rough time getting along lately.',
  'actual': 'Our code integration process needs to be reviewed for conflicts.'},
 {'vague': "Can someone ping the systems that aren't smooth anymore?",
  'actual': 'We should monitor the systems that are experiencing instability.'},
 {'vague': 'I think the environment needs a bit of a tune-up, right?',
  'actual': 'We need to stabilize our development and production environments.'},
 {'vague': 'Let’s not forget about the dragons we left unchecked in the server realm.',
  'actual': 'We have unresolved issues in our server configurations that need attention.'},
 {'vague': 'Can we sprinkl

In [21]:
for doc in data:
    doc['vague_embedding'] = model.encode(doc['vague'])
    concatenated_fields = doc['vague'] + doc['actual']
    doc['id'] = hashlib.md5(concatenated_fields.encode()).hexdigest()


In [24]:
index_settings = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 0
    },  
    'mappings': {
        'properties': {
            'id': {'type': 'keyword'},
            'vague': {'type': 'text'},
            'actual': {'type': 'text'},
            'vague_embedding': {'type': 'dense_vector', 
                                'dims': 384,
                                'index': True,
                                'similarity': 'cosine'}
        }
    }
}

In [25]:
index_name = 'vague-actual'
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vague-actual'})

In [26]:
for doc in tqdm(data):
    es_client.index(index=index_name, id=doc['id'], body=doc)

  0%|          | 0/413 [00:00<?, ?it/s]

In [27]:
query = 'Can we sprinkle some holy water on our deployment pipeline?'

In [28]:
v_q= model.encode(query)